# EO Point Cloud Generation  - Basic part III
## IGN HD LiDAR PCD Generation

This Notebook provide a basic processing pipeline to:
- Download IGN tile liDAR data 
- Read data with pdal and project data in UTM
- Assign custom color to the different classes
- Downsample data if needed
- Filter data for a given class
- Generate Point Cloud Data object with Open3D
- Save point cloud as ply object

## Import dependencies

In [1]:
import sys
import os
import geopandas as gpd
import random
import subprocess  # Used to execute external commands (e.g., wget)

# Add custom utility functions from the src directory
sys.path.append('../src/util/')
from general import IGNLidarProcessor, PointCloudHandler

## Open IGN LiDAR data information

In [2]:
# Define the path to the shapefile containing LiDAR metadata
shapefile_path = "../src/data/TA_diff_pkk_lidarhd_classe.shp"
# Load the shapefile into a GeoDataFrame
gdf = gpd.read_file(shapefile_path)
gdf.head()

,nom_pkk,url_telech,geometry
0,LHD_FXX_0230_6852_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((230000 6851000, 230000 6852000, 2310..."
1,LHD_FXX_0232_6870_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((232000 6869000, 232000 6870000, 2330..."
2,LHD_FXX_0187_6874_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((187000 6873000, 187000 6874000, 1880..."
3,LHD_FXX_0196_6839_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((196000 6838000, 196000 6839000, 1970..."
4,LHD_FXX_0229_6846_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((229000 6845000, 229000 6846000, 2300..."


In [3]:
# Directory to save the downloaded files
download_dir = '../src/data/ign'
# Create the download directory if it doesn't exist
os.makedirs(download_dir, exist_ok=True)

# Function to download a file using wget
def download_file(url, download_dir):
    """
    Downloads a file from a given URL using wget.

    Parameters:
        url (str): The URL of the file to download.
        download_dir (str): The directory where the file will be saved.

    Returns:
        str: Path to the downloaded file, or None if the download fails.
    """
    file_name = os.path.join(download_dir, os.path.basename(url))

    try:
        # Construct and execute the wget command
        subprocess.run(['wget', '-O', file_name, url], check=True)
        print(f"Downloaded: {file_name}")
        return file_name
    except subprocess.CalledProcessError as e:
        print(f"Error downloading {url}: {e}")
        return None

In [4]:
url = random.choice(gdf["url_telech"])
download_file(url=url, download_dir=download_dir)

--2025-03-19 18:49:51--  https://storage.sbg.cloud.ovh.net/v1/AUTH_63234f509d6048bca3c9fd7928720ca1/ppk-lidar/FR/LHD_FXX_0416_6203_PTS_C_LAMB93_IGN69.copc.laz
Resolving storage.sbg.cloud.ovh.net (storage.sbg.cloud.ovh.net)... 54.38.230.76
Connecting to storage.sbg.cloud.ovh.net (storage.sbg.cloud.ovh.net)|54.38.230.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130846290 (125M) [application/octet-stream]
Saving to: ‘../src/data/ign/LHD_FXX_0416_6203_PTS_C_LAMB93_IGN69.copc.laz’

     0K .......... .......... .......... .......... ..........  0% 2.32M 54s
    50K .......... .......... .......... .......... ..........  0% 4.74M 40s
   100K .......... .......... .......... .......... ..........  0% 4.50M 36s
   150K .......... .......... .......... .......... ..........  0%  139M 27s
   200K .......... .......... .......... .......... ..........  0%  184M 22s
   250K .......... .......... .......... .......... ..........  0% 4.81M 22s
   300K .......... .....

Downloaded: ../src/data/ign/LHD_FXX_0416_6203_PTS_C_LAMB93_IGN69.copc.laz


........ .......... .......... 96% 2.00M 0s
123100K .......... .......... .......... .......... .......... 96%  220M 0s
123150K .......... .......... .......... .......... .......... 96%  369M 0s
123200K .......... .......... .......... .......... .......... 96%  351M 0s
123250K .......... .......... .......... .......... .......... 96%  159M 0s
123300K .......... .......... .......... .......... .......... 96%  404M 0s
123350K .......... .......... .......... .......... .......... 96%  396M 0s
123400K .......... .......... .......... .......... .......... 96%  382M 0s
123450K .......... .......... .......... .......... .......... 96%  278M 0s
123500K .......... .......... .......... .......... .......... 96%  185M 0s
123550K .......... .......... .......... .......... .......... 96%  264M 0s
123600K .......... .......... .......... .......... .......... 96%  278M 0s
123650K .......... .......... .......... .......... .......... 96%  438M 0s
123700K .......... .......... .......... ...

'../src/data/ign/LHD_FXX_0416_6203_PTS_C_LAMB93_IGN69.copc.laz'

## Process the LiDAR Data


In [5]:
filename = os.path.join(download_dir, url.split('/')[-1])
sample_fraction = 0.2
# Initialize the LiDAR processor with the downloaded file
processor = IGNLidarProcessor(filename)
# Load LiDAR data
processor.read_point_cloud()
# Convert coordinates to UTM
processor.transform_coordinates()
# Apply colors based on class labels
processor.assign_custom_colors()

## Convert to Open3D and Save as PLY

In [6]:
object_filename = url.split('/')[-1].replace(".copc.laz",".ply")
# Initialize the point cloud handler
handler = PointCloudHandler(processor.df)
# Downsample and convert to Open3D forma
handler.downsample(sample_fraction=sample_fraction)
handler.to_open3d()
# Save the processed point cloud
handler.save_point_cloud(filename=object_filename)

Point cloud downsampled (fraction: 0.2)
Open3D PointCloud object created successfully.
Point cloud saved to LHD_FXX_0416_6203_PTS_C_LAMB93_IGN69.ply


End of the Notebook 🚀